In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import kplr
%matplotlib qt
import lightkurve as lk
import requests

In [2]:
def cut_time_series(time_series: pd.DataFrame, starting_transit:int, number_of_transits:int, T0: float, period: float) -> pd.DataFrame:
    number_of_transits = number_of_transits-starting_transit

    transit_start=(T0+(period*starting_transit))
    transit_end=(T0+(period*number_of_transits))
    
    df_cut = time_series.query(f'Time>={transit_start} & Time<={transit_end}')

    df_cut = df_cut.fillna(df_cut.mean())
    df_cut['Flux'] -= np.mean(df_cut['Flux'])
    return df_cut

In [3]:
def phase_fold(cut_time_series:pd.DataFrame, period:float ) -> pd.DataFrame:
    timestep = cut_time_series.index[2] - cut_time_series.index[1]
    ts_length = len(cut_time_series)
    ts=cut_time_series.index
    phase=np.mod((ts-ts.min())+period/2, period)/period
    phis = 2.*np.pi*np.mod((ts-ts.min())/(period),1) 
    cut_time_series['phis'] = phis
    cut_time_series['phase'] = phase
    return cut_time_series

In [57]:
def harmonics(cut_time_series:pd.DataFrame, period:float, number_of_transits):
    ts_length = len(cut_time_series)
    
    nharms = np.linspace(1,ts_length/number_of_transits,ts_length//number_of_transits)
    
    phis = cut_time_series['phis'].values
    pmat = nharms*phis.reshape(ts_length,1)
    
    C = np.cos(pmat)
    S = np.sin(pmat)

    Tmat = np.matrix(np.hstack((C,S)))
    
    I = cut_time_series['Flux'].values
    I_mat = np.matrix(I.reshape(ts_length,1))
    
    R = (Tmat.T*Tmat).I * (Tmat.T*I_mat)

    fit = np.array(Tmat*R).reshape(ts_length)
    dat = I.reshape(ts_length)

    # Now plot phase-folded
    phase = cut_time_series['phase'].values
    isort = np.argsort(phase)
    phase = phase[isort]
    fit = fit[isort]
    dat = dat[isort]
    return fit, dat, phase, R,nharms, Tmat

In [58]:
#harmonics(df_cut, period, number_of_transits)

In [59]:
kepler_list = pd.read_csv('kep_conf_names_2023_02_18_15_00_44.csv', delimiter=',')

In [60]:

def download_kepler(kepler_name):
    search_results = lk.search_lightcurve(kepler_name, author='Kepler', cadence="long")
    
    dfs_list = []
    for lc in search_results:
        light_curve = lc.download()
        normalised_flux = light_curve.flux.value / np.mean(light_curve.flux.value)
        df_temp = pd.DataFrame([light_curve.time.mjd,normalised_flux]).astype(float)
        dfs_list.append(df_temp)
    
    if len(dfs_list) == 0:
        print(f'No Data for {kepler_name}')
        return dfs_list

    
    df_temp2 = pd.concat(dfs_list, axis=1)
    

    return df_temp2

In [61]:
#for kepler_name in kepler_list['kepler_name']:
print('##################')
kepler_name = 'Kepler-1055 b'#'Kepler-14 b'
try:
    del period, T0, df
except:
    pass
print(kepler_name)
url = f"https://archive.stsci.edu/kepler/confirmed_planets/search.php?kepler_name={kepler_name.replace(' ','+')}&max_records=1&action=Search&outputformat=JSON&coordformat=dec&verb=3"
r = requests.get(url).json()[0]
period = float(r['Period'])
print(period)
T0 = 54965.19299999997#float(r['Time of transit'])

df = download_kepler(kepler_name)
if len(df)==0:
    print("No Lightcurve")
    #continue
df = df.T
df.columns = ['Time', 'Flux']
df = df.set_index('Time')

std_devs = 0
starting_transit = 0

#period = 6.79012361 # Kepler 14 b
#T0 = 138.088 # Kepler 14 b
number_of_transits = int((df.index[-1] - df.index[0])/period)-1
number_of_transits = 300 if number_of_transits>300 else number_of_transits

df_cut = cut_time_series(df, starting_transit, number_of_transits, T0, period)
#df_cut.plot( y='Flux')
df_cut = phase_fold(df_cut, period )
#df_cut.plot(x= 'phase', y='Flux')

ts_length = len(df_cut)

fit, dat, phase, R, nharms, Tmat = harmonics(df_cut, period, number_of_transits)


print(len(fit), len(dat), len(R), len(nharms))

# Inew=np.array(Tmat*R)
# print(len(Inew))
# plt.figure(4)
# plt.plot(phase,Inew,'.b', markersize = 2)
# plt.show()


outer_l4,inner_l4, inner_l5, outer_l5 = 0.24, 0.42, 0.57, 0.75
in_trojon_region_l4 = (phase>outer_l4) & (phase<inner_l4)
in_trojon_region_l5 = (phase>inner_l5) & (phase<outer_l5)

outside_trojan_and_planet = (phase<outer_l4) | (phase>outer_l5)

in_trojon_region = in_trojon_region_l4 | in_trojon_region_l5

mean_outside_trojan_region = np.mean(fit[outside_trojan_and_planet])
std_outside_trojan_region = np.std(fit[outside_trojan_and_planet])


if np.sum(fit[in_trojon_region]<mean_outside_trojan_region-std_outside_trojan_region*std_devs)>0:
    plt.figure(2)
    plt.plot(phase,fit,'r')
    plt.axhline(mean_outside_trojan_region)
    plt.axhline(mean_outside_trojan_region-std_outside_trojan_region*std_devs)
    plt.axvline(outer_l4)
    plt.axvline(inner_l4)
    plt.axvline(inner_l5)
    plt.axvline(outer_l5)
    plt.plot(phase,dat,',b')
    ddd
else:
    print('Nothing detected')

##################
Kepler-1055 b
2.295
30622 30622 204 102


NameError: name 'ddd' is not defined

In [18]:
df_cut.plot(y='Flux')

<AxesSubplot:xlabel='Time'>

In [9]:
df_test = df_cut.loc[df_cut.index[:100]]

In [33]:
ts_length = len(df_test)
print(ts_length)
    
nharms = np.linspace(int(1),int(ts_length/number_of_transits),int(ts_length//number_of_transits)).astype('float32')

print(1)

phis_reshaped = df_test['phis'].values.reshape(ts_length, 1)

pmat = nharms*phis_reshaped
print(1)
C = np.cos(pmat)
S = np.sin(pmat)
Tmat = np.matrix(np.hstack((C,S)))
print(1)

100
1
1
1


In [41]:
np.shape(Tmat)

(100, 32)

In [61]:
print(1)
I = df_test['Flux'].values
I_mat = df_test['Flux'].values.reshape(ts_length,1)
print(1)
R = (Tmat.T*Tmat).I * (Tmat.T*I_mat)

1
1


In [62]:
np.shape(R)

(32, 1)

In [188]:

df.plot()

<AxesSubplot:xlabel='Time'>

51402 51402 474 237
Nothing detected


In [90]:


#plt.show()

#print(np.shape(R))
#R = np.array(R).reshape(NCPT,1)

plt.figure(2)
plt.step(nharms/number_of_transits,R[:int(ts_length/number_of_transits)],'.b',where='mid',label="Cosine")
plt.step(nharms/number_of_transits,R[int(ts_length/number_of_transits):],'.r',where='mid',label="Sine")

# plt.figure(3)
# plt.step(nharms,R[:int(ts_length/number_of_transits)],label="Cosine")
# plt.step(nharms,R[int(ts_length/number_of_transits):],'.r',label="Sine")
plt.legend()
#plt.xlim(0,30)
#plt.ylim(-1.5e-4,0.5e-4)
#plt.step(nharms,R[:NCPT],'.b',where='mid')
#plt.step(nharms,R[NCPT:],'.r',where='mid')



1306
